In [1]:
import numpy as np
import pprint as pp

In [2]:
class LinearProgram:
    def __init__(self, A: np.ndarray, b: np.ndarray, c: np.ndarray):
        n = A.shape[0]
        self.A = np.concatenate([A, np.eye(n)], axis=1)
        # self.b = np.concatenate([b, np.zeros(n)])
        self.b = b
        self.c = np.concatenate([c, np.zeros(n)])
        
    def __repr__(self):
        return pp.pformat({"A": self.A, "b": self.b, "c": self.c})

In [3]:
def get_vertex(B: np.ndarray, lp: 'LinearProgram'):
    A, b, c = lp.A, lp.b, lp.c
    b_inds = np.sort(B)
    AB = A[:, b_inds]
    b = b[b_inds]
    xb = np.linalg.solve(AB, b)
    x = np.zeros_like(c)
    x[b_inds] = xb
    return x

In [4]:
A = np.array([ [-1.0, -1.0, 0.0, 1.0], [1.0, -1.0, -1.0, -2.0] ])
b = np.array([ 1.0, -2.0, 0.0, 4.0 ])
c = np.array([ 2.0, 1.0 ])
lp = LinearProgram(A, b, c)

In [5]:
B = np.array([1, 2])
get_vertex(B, lp)

array([ 0.,  2., -2.,  0.])

In [6]:
def edge_transition(lp: 'LinearProgram', B: np.ndarray, q: int):
    A, b, c = lp.A, lp.b, lp.c
    n = A.shape[1]
    b_inds = np.sort(B)
    n_inds = np.array([i for i in range(n) if i not in b])
    AB = A[: , b_inds]
    xB = np.linalg.solve(AB, b)
    d = np.linalg.solve(AB, A[:, n_inds[q]])
    p, xqp = 0, np.inf
    
    for i in range(len(d)):
        if d[i] > 0:
            v = xB[i] / d[i]
            if v < xqp:
                p, xqp = i, v
    
    return (p, xqp)

In [7]:
A = np.array([ [1, 1], [-4, 2] ])
b = np.array([ 9, 2 ])
c = np.array([ 3, -1 ])
lp = LinearProgram(A, b, c)
B = np.array([2, 3])
q = 1
edge_transition(lp, B, q)

(1, 1.0)

In [8]:
def step_lp(lp: 'LinearProgram', B: np.ndarray):
    A, b, c = lp.A, lp.b, lp.c
    n = A.shape[1]
    b_inds = np.sort(B)
    n_inds = np.array([i for i in range(n) if (i not in b_inds)])
    print(b_inds)
    print(n_inds)
    AB = A[: , b_inds]
    AV = A[:, n_inds]
    xB = np.linalg.solve(AB, b)
    cB = c[b_inds]
    lam = np.linalg.solve(AB.T, cB)
    cV = c[n_inds]
    muV = cV - AV.T @ lam

    print(xB)
    print(lam)
    print(muV)
    
    q, p, xqp, delta = 0, 0, np.inf, np.inf
    
    for i in range(len(muV)):
        if muV[i] < 0:
            pi, xip = edge_transition(lp, B, i)
            if muV[i] * xip < delta:
                q, p, xqp, delta = i, pi, xip, muV[i] * xip

    if q == 0:
        return (B, True)
    
    if xqp == np.inf:
        raise(ValueError("Unbounded"))
    
    j = B.tolist().index(b_inds[p])
    B[j] = n_inds[q]
    
    return (B, False)

In [9]:
def minimize_lp(B, lp):
    done = False
    
    while not done:
        B, done = step_lp(lp, B)
    
    return B

In [10]:
A = np.array([ [1, 1], [-4, 2] ])
b = np.array([ 9, 2 ])
c = np.array([ 3, -1 ])
lp = LinearProgram(A, b, c)
B = np.array([2, 3])

B  = minimize_lp(B, lp)

[2 3]
[0 1]
[9. 2.]
[0. 0.]
[ 3. -1.]
[1 2]
[0 3]
[1. 8.]
[ 0.  -0.5]
[1.  0.5]


In [11]:
B

array([2, 1])

In [12]:
lp.A[: , B]

array([[1., 1.],
       [0., 2.]])

In [13]:
lp.A[: , [1, 2]]

array([[1., 1.],
       [2., 0.]])

In [14]:
AB = lp.A[: , [1, 2]]
np.linalg.solve(AB, b)

array([1., 8.])

In [15]:
A = np.array([ [3.0, 2.0, 1.0], [2.0, 5.0, 3.0] ])
b = np.array([ 10.0, 15.0 ])
c = np.array([ -2.0, -3.0, -4.0])

lp = LinearProgram(A, b, c)
B = np.array([3, 4])

B  = minimize_lp(B, lp)
B

[3 4]
[0 1 2]
[10. 15.]
[0. 0.]
[-2. -3. -4.]
[2 3]
[0 1 4]
[5. 5.]
[ 0.         -1.33333333]
[0.66666667 3.66666667 1.33333333]


array([3, 2])

In [16]:
AB = lp.A[: , B]
np.linalg.solve(AB, b)

array([5., 5.])

In [17]:
x = np.zeros(lp.A.shape[1])
print(x)
x[B] = np.linalg.solve(AB, b)
print(x[:3])

[0. 0. 0. 0. 0.]
[0. 0. 5.]


In [18]:
x[:3].dot(c)

-20.0

In [19]:
import pulp

# Create a linear programming problem (minimization)
lp_problem = pulp.LpProblem("MinimizationProblem", pulp.LpMinimize)

# Define variables (assuming they are non-negative)
x = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(len(c))]

# Set the objective function
lp_problem += pulp.lpDot(c, x), "ObjectiveFunction"

# Add constraints
for i in range(len(b)):
    lp_problem += (pulp.lpDot(A[i], x) <= b[i], f"Constraint_{i+1}")

# Solve the problem
lp_problem.solve()

# Print the results
print("Status:", pulp.LpStatus[lp_problem.status])
print("Objective value:", pulp.value(lp_problem.objective))
for i, var in enumerate(x):
    print(f"x{i} = {var.value()}")


Status: Optimal
Objective value: -20.0
x0 = 0.0
x1 = 0.0
x2 = 5.0
